In [16]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from uuid import UUID

import matplotlib.pyplot as plt

import confusion_matrix_handling as cm_handling
from confusion_matrix_handling import MODE_MAPPING_DICT
import get_EC
import helper_functions as hf

import sklearn.model_selection as skm

from sklearn.model_selection import KFold
from sklearn import linear_model

METERS_TO_MILES = 0.000621371 # 1 meter = 0.000621371 miles
ECAR_PROPORTION = 0 #0.01 #~1% of cars on the road are electric.
DROVE_ALONE_TO_SHARED_RIDE_RATIO = 1

df_EI = pd.read_csv(r'Public_Dashboard/auxiliary_files/energy_intensity.csv') # r stands for raw string, only matters if the path is on Windows

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Run store_expanded_labeled_trips instead
'''import database_related_functions as drf  # all the emission server functions for this notebook are in here.
user_list, os_map, uuid_program_map = drf.get_participants_programs_and_operating_systems()
#print(len(user_list), len(os_map), len(uuid_program_map))

# Takes 6 to 14 minutes for the full 1.5 year CEO study plus stage and prepilot.
# Takes ~ 1 min 45 s to 2 min 45 s on Macbook Pro for all ceo data up to May 2022.
expanded_labeled_trips = drf.get_expanded_labeled_trips(user_list)
expanded_labeled_trips['os'] = expanded_labeled_trips.user_id.map(os_map)
expanded_labeled_trips['program'] = expanded_labeled_trips['user_id'].map(uuid_program_map)

expanded_labeled_trips = expanded_labeled_trips.drop(labels = ['source', 'end_fmt_time', 'end_loc', 'raw_trip',
    'start_fmt_time', 'start_loc','start_local_dt_year', 'start_local_dt_month', 'start_local_dt_day',
    'start_local_dt_hour', 'start_local_dt_minute', 'start_local_dt_second',
    'start_local_dt_weekday', 'start_local_dt_timezone',
    'end_local_dt_year', 'end_local_dt_month', 'end_local_dt_day',
    'end_local_dt_hour', 'end_local_dt_minute', 'end_local_dt_second',
    'end_local_dt_weekday', 'end_local_dt_timezone'], axis = 1)

expanded_labeled_trips['distance_miles'] = expanded_labeled_trips.distance*METERS_TO_MILES

# Group together the prepilot participants
prepilot_list = ['84Q9SsrH','cwZazZLJ','CudLAeg8','sxxcLqbK','Q8T7QTXK','5KEGHHuf','e9MaNVU7','7c797MRD','rhBZukxY','k36cxmfA','FmxVf8u6','F3jxHLSW']
expanded_labeled_trips['program'] = expanded_labeled_trips.program.replace(prepilot_list, "prepilot")'''

'import database_related_functions as drf  # all the emission server functions for this notebook are in here.\nuser_list, os_map, uuid_program_map = drf.get_participants_programs_and_operating_systems()\n#print(len(user_list), len(os_map), len(uuid_program_map))\n\n# Takes 6 to 14 minutes for the full 1.5 year CEO study plus stage and prepilot.\n# Takes ~ 1 min 45 s to 2 min 45 s on Macbook Pro for all ceo data up to May 2022.\nexpanded_labeled_trips = drf.get_expanded_labeled_trips(user_list)\nexpanded_labeled_trips[\'os\'] = expanded_labeled_trips.user_id.map(os_map)\nexpanded_labeled_trips[\'program\'] = expanded_labeled_trips[\'user_id\'].map(uuid_program_map)\n\nexpanded_labeled_trips = expanded_labeled_trips.drop(labels = [\'source\', \'end_fmt_time\', \'end_loc\', \'raw_trip\',\n    \'start_fmt_time\', \'start_loc\',\'start_local_dt_year\', \'start_local_dt_month\', \'start_local_dt_day\',\n    \'start_local_dt_hour\', \'start_local_dt_minute\', \'start_local_dt_second\',\n    \

In [18]:
%store -r expanded_labeled_trips

In [19]:
unit_dist_MCS_df = pd.read_csv("unit_distance_MCS.csv").set_index("moment")
energy_dict = cm_handling.get_energy_dict(df_EI, units='MWH')

In [20]:
expanded_labeled_trips = hf.drop_unwanted_trips(expanded_labeled_trips,drop_not_a_trip=False)
expanded_labeled_trips = hf.get_primary_modes(expanded_labeled_trips,energy_dict,MODE_MAPPING_DICT)
print('Here are the number of labeled trips remaining in each program dataset:')
expanded_labeled_trips.program.value_counts()

Dropping user labeled AIR trips and trips with no OS.
Dropped 93 trips with no sensed sections.
Here are the number of labeled trips remaining in each program dataset:


cc          28768
pc          17880
fc          11744
stage       10715
sc           9092
vail         6348
4c           5262
prepilot     2425
Name: program, dtype: int64

In [21]:
# Get the confusion matrices and then the EI moments from those.
android_confusion = pd.read_csv("android_confusion.csv").set_index('gt_mode')
ios_confusion = pd.read_csv("ios_confusion.csv").set_index('gt_mode')

android_confusion = cm_handling.collapse_confusion_matrix(android_confusion, rows_to_collapse={"Train": ["Train"]}, columns_to_collapse={})
ios_confusion = cm_handling.collapse_confusion_matrix(ios_confusion, rows_to_collapse={"Train": ["Train"]}, columns_to_collapse={})

# here I'm referring to car_load_factor the number that we divide the drove alone energy intensity by
# for r = 1, car_load_factor is 4/3.
sensed_car_EI = hf.find_sensed_car_energy_intensity(energy_dict, ECAR_PROPORTION, DROVE_ALONE_TO_SHARED_RIDE_RATIO)
energy_dict.update({"Car, sensed": sensed_car_EI})
expanded_labeled_trips['distance_miles'] = expanded_labeled_trips.distance*METERS_TO_MILES
EI_length_cov = 0

In [91]:
# note. Approach differently if the android ground truth modes are not the same set as ios ground truth modes in the test dataset.
available_ground_truth_modes = android_confusion.index

# Previously, with duration CMs, I included no_gt. 
# However, since no_gt is zero in the trajectory distance CMs, I set all no_gt priors to zero.
uniform_prior = hf.construct_prior_dict({"no_gt": 0},available_ground_truth_modes)
mostly_car_prior = hf.construct_prior_dict({"Car, sensed": 0.80, "Pilot ebike": 0.05, "no_gt": 0}, available_ground_truth_modes)
mostly_ebike_prior = hf.construct_prior_dict({"Pilot ebike": 0.80, "Pilot ebike": 0.05, "no_gt": 0}, available_ground_truth_modes)
mostly_train_prior = hf.construct_prior_dict({"Train": 0.30, "no_gt": 0}, available_ground_truth_modes)
half_car = hf.construct_prior_dict({"Car, sensed": 0.50, "no_gt": 0}, available_ground_truth_modes)
half_ebike = hf.construct_prior_dict({"Pilot ebike": 0.50, "no_gt": 0}, available_ground_truth_modes)
close_to_CEO_true_car_and_ebike = hf.construct_prior_dict({"Car, sensed": 0.7, "Pilot ebike": 0.13,"no_gt": 0}, available_ground_truth_modes)



prior_mode_distributions_map = {
    "80 Percent Car": mostly_car_prior,
    "80 Percent Ebike": mostly_ebike_prior,
    "30 Percent Train": mostly_train_prior,
    "50 Percent Car": half_car,
    "50 Percent Ebike": half_ebike,
    "No Bayes Update": {},
    "MobilityNet Specific to OS": {},
    "Uniform Prior": uniform_prior,
    "Close to CEO Mode Distribution": close_to_CEO_true_car_and_ebike,
}
#prior_and_error_dataframe_vail,_  = hf.prior_mode_distribution_sensitivity_analysis(expanded_labeled_trips[expanded_labeled_trips.program == 'vail'], prior_mode_distributions_map, android_confusion, ios_confusion, unit_dist_MCS_df, energy_dict, EI_length_cov)

In [94]:
prior_and_error_dataframe,_  = hf.prior_mode_distribution_sensitivity_analysis(
    expanded_labeled_trips, 
    prior_mode_distributions_map, 
    android_confusion, 
    ios_confusion, 
    unit_dist_MCS_df, 
    energy_dict, 
    EI_length_cov)

80 Percent Car
Computing energy consumption for each trip.
Using EI length covariance = 0.
80 Percent Ebike
Computing energy consumption for each trip.
Using EI length covariance = 0.
30 Percent Train
Computing energy consumption for each trip.
Using EI length covariance = 0.
50 Percent Car
Computing energy consumption for each trip.
Using EI length covariance = 0.
50 Percent Ebike
Computing energy consumption for each trip.
Using EI length covariance = 0.
No Bayes Update
Computing energy consumption for each trip.
Using EI length covariance = 0.
MobilityNet Specific to OS
Computing energy consumption for each trip.
Using EI length covariance = 0.
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
Close to CEO Mode Distribution
Computing energy consumption for each trip.
Using EI length covariance = 0.


### Priors for paper

In [95]:
priors_for_paper = ["80 Percent Car", "No Bayes Update", "MobilityNet Specific to OS", "Uniform Prior"]
print(prior_and_error_dataframe.query('`Prior Name` == @priors_for_paper').round(2).to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
                 Prior Name &  Percent Error &  Estimated Standard Deviation (SD) &  Number of Standard Deviations to Truth \\
\midrule
             80 Percent Car &          21.57 &                             136.52 &                                    0.74 \\
            No Bayes Update &          13.96 &                             152.42 &                                    0.43 \\
 MobilityNet Specific to OS &          13.96 &                             152.42 &                                    0.43 \\
              Uniform Prior &          15.70 &                             153.73 &                                    0.48 \\
\bottomrule
\end{tabular}



In [96]:
prior_and_error_dataframe

,Prior Name,Percent Error,Estimated Standard Deviation (SD),Number of Standard Deviations to Truth
0,80 Percent Car,21.573168,136.518450,0.743791
1,80 Percent Ebike,15.773328,153.749822,0.482878
2,30 Percent Train,11.905138,153.743555,0.364474
3,50 Percent Car,18.851997,140.183538,0.632978
4,50 Percent Ebike,15.315175,153.653480,0.469146
5,No Bayes Update,13.962821,152.422821,0.431173
6,MobilityNet Specific to OS,13.962821,152.422821,0.431173
7,Uniform Prior,15.696178,153.732361,0.480570
8,Close to CEO Mode Distribution,20.904255,136.864670,0.718905


In [97]:
prior_and_error_dataframe_cc,_  = hf.prior_mode_distribution_sensitivity_analysis(expanded_labeled_trips[expanded_labeled_trips.program == "cc"], prior_mode_distributions_map, android_confusion, ios_confusion, unit_dist_MCS_df, energy_dict, EI_length_cov)

80 Percent Car
Computing energy consumption for each trip.
Using EI length covariance = 0.
80 Percent Ebike
Computing energy consumption for each trip.
Using EI length covariance = 0.
30 Percent Train
Computing energy consumption for each trip.
Using EI length covariance = 0.
50 Percent Car
Computing energy consumption for each trip.
Using EI length covariance = 0.
50 Percent Ebike
Computing energy consumption for each trip.
Using EI length covariance = 0.
No Bayes Update
Computing energy consumption for each trip.
Using EI length covariance = 0.
MobilityNet Specific to OS
Computing energy consumption for each trip.
Using EI length covariance = 0.
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
Close to CEO Mode Distribution
Computing energy consumption for each trip.
Using EI length covariance = 0.


In [98]:
# Uniform priors for each program
prior_mode_distributions_map = {"Uniform Prior": uniform_prior}
uniform_prior_dataframe_map = {}
for program in expanded_labeled_trips.program.unique():
    print(program)
    uniform_prior_df, _ = hf.prior_mode_distribution_sensitivity_analysis(
        expanded_labeled_trips[expanded_labeled_trips.program == program], 
        prior_mode_distributions_map, android_confusion, ios_confusion, 
        unit_dist_MCS_df, energy_dict, EI_length_cov
        )
    uniform_prior_df["program"] = program
    uniform_prior_dataframe_map[program] = uniform_prior_df
uniform_prior_df_all_programs = pd.concat(uniform_prior_dataframe_map, ignore_index=True)

sc
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
fc
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
4c
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
stage
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
pc
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
vail
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
cc
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.
prepilot
Uniform Prior
Computing energy consumption for each trip.
Using EI length covariance = 0.


In [99]:
# these percent errors seem similar to those we get with the predicted values and no uncertainty.
uniform_prior_df_all_programs.set_index('program')

,Prior Name,Percent Error,Estimated Standard Deviation (SD),Number of Standard Deviations to Truth
program,,,,
sc,Uniform Prior,10.515667,7.220534,0.549044
fc,Uniform Prior,19.837382,8.856299,0.868093
4c,Uniform Prior,3.272010,9.946720,0.078192
stage,Uniform Prior,6.030970,21.553792,0.209436
pc,Uniform Prior,30.909686,29.960071,0.990314
vail,Uniform Prior,11.251835,14.461729,0.253345
cc,Uniform Prior,13.528741,62.793950,0.333913
prepilot,Uniform Prior,21.816170,3.262432,0.807205


In [100]:
durations_in_modes = android_confusion.sum(axis=1)
prior_mode_probs = durations_in_modes/android_confusion.sum().sum()


# for each prior: compute energy consumption.
android_EI_moments_df = cm_handling.get_conditional_EI_expectation_and_variance(android_confusion,energy_dict)
ios_EI_moments_df = cm_handling.get_conditional_EI_expectation_and_variance(ios_confusion,energy_dict)
os_EI_moments_map = {'ios': ios_EI_moments_df, 'android': android_EI_moments_df}
prior_mode_probs

gt_mode
Walk             0.027579
Regular Bike     0.009059
Pilot ebike      0.030208
Scooter share    0.013426
Car, sensed      0.082845
Bus              0.141820
no_gt            0.000000
Train            0.695063
dtype: float64